## Callin Switzer
## Run Simulations to compare output to NNET

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
import pandas as pd
import multiProcTraj # note that this is a custom-written file 
from multiprocessing import Pool, cpu_count


# define directories
baseDir = os.getcwd()
dataDir = r'D:\MothSimulations\11c-AggressiveManeuver\Qstore\hws_am_con'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
randomRawData = r'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/PythonGeneratedData'

if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)

print(sys.version)
now = datetime.now()
print("last run on " + str(now))

3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 11:48:23) [MSC v.1900 64 bit (AMD64)]
last run on 2019-01-01 13:25:00.847774


In [2]:
# Load data
testDF = pd.read_csv(os.path.join(dataOutput, "NNpreds_RandomICs.csv"))
testDF_sm= pd.read_csv(os.path.join(dataOutput, "NNpreds_small_RandomICs.csv"))

In [3]:
print(testDF_sm.shape)
testDF_sm.head()

(1000, 19)


,x_0,y_0,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,F_pred,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred
0,0.0,0.0,5.717836,2.903536,1140.923218,-399.095208,-20.919668,7.162696,5242.905,0.227884,87642.836,21.425903,-8.116010,5.438490,2.756136,966.60364,-434.46356,-11.772325,-5.640531
1,0.0,0.0,3.799657,4.295602,1216.867455,1286.049934,9.030914,5.734873,21598.404,2.597742,-67889.390,27.849758,27.358930,3.870090,4.277221,1548.62660,1466.32930,-1.445739,-6.679240
2,0.0,0.0,1.426920,3.170749,1008.157261,175.219341,-19.672703,-17.241171,23363.543,1.863295,-15468.784,20.439296,-0.898106,1.004533,2.738070,1000.94440,-266.57200,-23.733850,-25.134838
3,0.0,0.0,1.401717,4.076364,-1022.919335,-980.612881,-9.282598,-20.555771,32194.880,3.678808,68082.640,-19.039632,-13.359467,1.199188,3.981795,-955.95240,-369.71906,-4.945975,0.057465
4,0.0,0.0,0.350069,5.063611,827.387703,1371.685241,2.420546,-2.941864,15569.313,3.173099,-8812.373,15.881660,29.669649,0.238542,4.952067,745.15857,1623.83780,-10.901551,-11.416786


In [4]:
testDF_sm[["x_0", "x_dot_0", "y_0", "y_dot_0", 
           "theta_0", "theta_dot_0", "phi_0", "phi_dot_0", 
           "F_pred", "alpha_pred", "tau_pred"]].head()

,x_0,x_dot_0,y_0,y_dot_0,theta_0,theta_dot_0,phi_0,phi_dot_0,F_pred,alpha_pred,tau_pred
0,0.0,1140.923218,0.0,-399.095208,2.903536,7.162696,5.717836,-20.919668,5242.905,0.227884,87642.836
1,0.0,1216.867455,0.0,1286.049934,4.295602,5.734873,3.799657,9.030914,21598.404,2.597742,-67889.390
2,0.0,1008.157261,0.0,175.219341,3.170749,-17.241171,1.426920,-19.672703,23363.543,1.863295,-15468.784
3,0.0,-1022.919335,0.0,-980.612881,4.076364,-20.555771,1.401717,-9.282598,32194.880,3.678808,68082.640
4,0.0,827.387703,0.0,1371.685241,5.063611,-2.941864,0.350069,2.420546,15569.313,3.173099,-8812.373


In [6]:
testDF.shape[0]


1000000

___

# Loop Version (Not used -- see parallel version below)

In [8]:
# put data into correct order
# x,xd,y,yd,
# theta,thetad,phi,phid, 
# F, alpha, tau0


tic = time.time()
t = np.linspace(0, 0.02, num = 100, endpoint = False) # time cut into 100 timesteps
nrun = testDF.head(1000).shape[0]  #number of trajectories.

# pre-allocate arrays to fill
nstep = 100
zeroMatrix = np.zeros([nrun, nstep])
x,      xd,    y,   yd, \
theta, thetad, phi, phid = [zeroMatrix.copy() for ii in 
                                range(len([ "x",     "xd",     "y", "yd", 
                                            "theta", "thetad", "phi", "phid"]))]

# generate random initial conditions for state 0
state0 = np.array(testDF[["x_0", "x_dot_0", "y_0", "y_dot_0", 
           "theta_0", "theta_dot_0", "phi_0", "phi_dot_0", 
           "F_pred", "alpha_pred", "tau_pred"]])

# loop through all the runs
for i in range(0,nrun):
    # run ODE
    state = odeint(multiProcTraj.FlyTheBug, state0[i, :], t)
    x[i,:], xd[i,:] = state[:,0], state[:,1]
    y[i,:], yd[i, :] = state[:,2], state[:,3]
    theta[i,:], thetad[i, :] = state[:,4],state[:,5]
    phi[i,:], phid[i, :] = state[:, 6], state[:,7]
    
    if(np.mod(i, 1000) == 0):
        print(i)
        print('elapsed time = ',time.time()-tic)
print('elapsed time = ',time.time()-tic)

0
elapsed time =  0.2952103614807129
elapsed time =  44.22075033187866


In [ ]:
x.shape

In [ ]:
state0.shape

In [ ]:
plt.figure()
for i in range(0,nrun):
    plt.plot(x[i, :],y[i, :], label = 'trajectory x vs y')

In [ ]:
# get final states
x_f_sim = x[:, -1]
xd_f_sim = xd[:, -1]

y_f_sim = y[:, -1]
yd_f_sim = yd[:, -1]

theta_f_sim = theta[:, -1]
thetad_f_sim = thetad[:, -1]

phi_f_sim = phi[:, -1]
phid_f_sim = phid[:, -1]

In [ ]:
combDF = testDF.head(100000).copy()

In [ ]:
combDF = combDF.assign(x_99_sim = x_f_sim, 
                      y_99_sim = y_f_sim, 
                      theta_99_sim = theta_f_sim, 
                      phi_99_sim = phi_f_sim, 
                      xd_99_sim = xd_f_sim, 
                      yd_99_sim = yd_f_sim, 
                      thetad_99_sim = thetad_f_sim, 
                      phid_99_sim = phid_f_sim) 

In [ ]:
combDF.head()

In [ ]:
combDF.to_csv(os.path.join(dataOutput, "SimVsNNet_Locations_100K.csv"), index = False)

___

# parallel version

The data is read in in multiProcTraj.py

In [43]:
# put data into correct order
# x,xd,y,yd,
# theta,thetad,phi,phid, 
# F, alpha, tau0

nrun = 1000000

tic = time.time()
# loop through all the runs

p = Pool(cpu_count()-3)
stt = time.time()   
bb = p.map(multiProcTraj.flyBug_firstLast_test, range(nrun) ) 
print(time.time() -stt)

p.close()
p.join()
print(time.time() -stt)
newDF = pd.DataFrame(bb, columns= ["x0", "x_99_sim", "xd0", "xd_99_sim", "y0", "y_99_sim", "yd0", "yd_99_sim", "theta0", \
        "theta_99_sim", "thetad0", "thetad_99_sim", "phi0", "phi_99_sim", "phid0", "phid_99_sim", "F", "alpha", "tau0"])
print('elapsed time = ',time.time()-tic)

3919.1579055786133
3919.639892578125
elapsed time =  3923.0148940086365


In [44]:
print(newDF.shape)

newDF.head()

(1000000, 19)


,x0,x_99_sim,xd0,xd_99_sim,y0,y_99_sim,yd0,yd_99_sim,theta0,theta_99_sim,thetad0,thetad_99_sim,phi0,phi_99_sim,phid0,phid_99_sim,F,alpha,tau0
0,0.0,9.613076,541.507416,448.015250,0.0,6.992493,502.568101,217.973019,6.272284,6.595571,-17.699701,24.360015,0.450877,0.715889,14.035800,21.315957,13372.245,4.137534,41779.530
1,0.0,23.115572,1409.737698,906.375367,0.0,23.024517,995.727369,1305.785387,2.831231,3.052383,19.846242,6.695834,1.913727,2.208943,17.112298,10.496111,32013.242,5.798155,-53341.395
2,0.0,11.656873,880.949432,293.894537,0.0,1.935931,178.694952,31.138036,3.740158,3.384689,-20.840242,-25.724889,1.081307,0.826801,-1.641278,-20.591758,31445.936,5.988265,-71866.240
3,0.0,-33.173762,-1401.856110,-1951.485267,0.0,-30.310488,-1363.320818,-1676.329486,3.084163,2.990945,5.480075,2.891852,4.572172,4.441047,-20.266189,0.998683,33695.890,0.617956,27833.310
4,0.0,25.656650,1420.324857,1178.470803,0.0,13.452704,545.096025,816.047812,4.596070,4.458516,-14.273627,-5.375318,5.882793,5.799361,-3.849331,-2.628247,19619.203,3.952550,-39307.790


In [45]:
# align newDF with old DF
testDF.head()

,x_0,y_0,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,F_pred,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred
0,0.0,0.0,0.450877,6.272284,541.507416,502.568101,14.035800,-17.699701,13372.245,4.137534,41779.530,9.347785,7.688051,0.654225,6.544707,431.10074,286.79320,18.787770,21.807661
1,0.0,0.0,1.913727,2.831231,1409.737698,995.727369,17.112298,19.846242,32013.242,5.798155,-53341.395,23.603876,23.633075,2.267903,3.124010,881.63740,1388.70240,17.693207,13.812419
2,0.0,0.0,1.081307,3.740158,880.949432,178.694952,-1.641278,-20.840242,31445.936,5.988265,-71866.240,11.786005,2.540601,0.827989,3.401310,272.30856,80.47675,-20.054775,-25.380537
3,0.0,0.0,4.572172,3.084163,-1401.856110,-1363.320818,-20.266189,5.480075,33695.890,0.617956,27833.310,-33.335655,-30.535096,4.439168,2.988790,-1944.18580,-1672.59780,1.120616,2.428478
4,0.0,0.0,5.882793,4.596070,1420.324857,545.096025,-3.849331,-14.273627,19619.203,3.952550,-39307.790,26.205825,13.510248,5.767939,4.434904,1192.81870,851.12103,-4.884992,-7.559634


In [46]:
combDF = newDF.merge(testDF.head(nrun), left_on=["F", "alpha", "tau0"], right_on=["F_pred", "alpha_pred", "tau_pred"])

In [47]:
print(combDF.shape)
combDF.head()

(1000000, 38)


,x0,x_99_sim,xd0,xd_99_sim,y0,y_99_sim,yd0,yd_99_sim,theta0,theta_99_sim,...,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred
0,0.0,9.613076,541.507416,448.015250,0.0,6.992493,502.568101,217.973019,6.272284,6.595571,...,4.137534,41779.530,9.347785,7.688051,0.654225,6.544707,431.10074,286.79320,18.787770,21.807661
1,0.0,23.115572,1409.737698,906.375367,0.0,23.024517,995.727369,1305.785387,2.831231,3.052383,...,5.798155,-53341.395,23.603876,23.633075,2.267903,3.124010,881.63740,1388.70240,17.693207,13.812419
2,0.0,11.656873,880.949432,293.894537,0.0,1.935931,178.694952,31.138036,3.740158,3.384689,...,5.988265,-71866.240,11.786005,2.540601,0.827989,3.401310,272.30856,80.47675,-20.054775,-25.380537
3,0.0,-33.173762,-1401.856110,-1951.485267,0.0,-30.310488,-1363.320818,-1676.329486,3.084163,2.990945,...,0.617956,27833.310,-33.335655,-30.535096,4.439168,2.988790,-1944.18580,-1672.59780,1.120616,2.428478
4,0.0,25.656650,1420.324857,1178.470803,0.0,13.452704,545.096025,816.047812,4.596070,4.458516,...,3.952550,-39307.790,26.205825,13.510248,5.767939,4.434904,1192.81870,851.12103,-4.884992,-7.559634


In [49]:
# save combined DF
combDF.to_csv(os.path.join(dataOutput, "SimVsNNet_Locations_Full.csv"), index = False)